## Integrate MERFISH experiments after QC filtering

In [ ]:
import scanpy as sc
import scvi
from tqdm.notebook import tqdm
import os
import numpy as np
import pandas as pd
from scvi.model.utils import mde
import matplotlib.pyplot as plt
import pickle
import glob

In [ ]:
sc.set_figure_params(figsize=(4, 4))

Add the paths to all folders for experiments to be integrated

In [ ]:
input_folders = glob.glob("/mnt/sata1/Analysis_Alex/human_r1/human_0*")

In [ ]:
output_folder = r"/mnt/sata1/Analysis_Alex/human_r1/analysis/cleaned"

Run scvi integration

In [ ]:
adata_list = []
for input_file in tqdm(input_folders):
    experiment = sc.read(
        os.path.join(input_file, "adatas", "04_axes_in_preparation.h5ad")
    )
    experiment.obs["batch"] = os.path.basename(input_file)
    adata_list.append(experiment.copy())

concatenated = sc.concat(adata_list)

concatenated.obs_names_make_unique()
concatenated.layers["counts"] = concatenated.X
scvi.model.SCVI.setup_anndata(concatenated, layer="counts", batch_key="batch")
vae = scvi.model.SCVI(concatenated, n_layers=2, n_latent=30, gene_likelihood="nb")
vae.train()

Use integration to calculate new clustering and neighbors

In [ ]:
concatenated.obsm["X_scVI"] = vae.get_latent_representation()

In [ ]:
sc.pp.neighbors(concatenated, use_rep="X_scVI")

Use MDE as new visualization

In [ ]:
concatenated.obsm["X_mde"] = mde(concatenated.obsm["X_scVI"])

In [ ]:
concatenated.obs["new_batch"] = concatenated.obs.batch.values

In [ ]:
xcoords = concatenated.obsm["X_mde"][:, 0]
ycoords = concatenated.obsm["X_mde"][:, 1]
category_names = concatenated.obs.new_batch.values

In [ ]:
sc.tl.leiden(concatenated)

In [ ]:
concatenated.write(os.path.join(output_folder, "concatenated_integrated.h5ad"))

In [ ]:
sc.set_figure_params(figsize=(10, 5))
sc.pl.embedding(concatenated, basis="mde", color=["leiden"])

## Save our SCVI VAE

In [ ]:
import pickle

# File path where you want to save the object
file_path = os.path.join(output_folder, "vae_object.pkl")

# Dump the object to the file
with open(file_path, "wb") as file:
    pickle.dump(vae, file)